In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#!pip category_encoders
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns; sns.set()
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn import ensemble
from sklearn.metrics import accuracy_score
import scipy.stats as stats
from sklearn.feature_extraction.text import TfidfVectorizer
#import category_encoders as ce
import warnings
import gc
import psutil 
from lightgbm import LGBMClassifier
warnings.filterwarnings('ignore')
%matplotlib inline

os.chdir('/content/drive/My Drive/ML/FinalProject/VEFFinalProject')
os.chdir('predict-stock')
#os.chdir('dataset')
!ls dataset

ls: cannot access 'dataset': No such file or directory


In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
market_dt = pd.read_csv('/content/drive/My Drive/ML/FinalProject/dataset/market_train_uni1_v1.csv')




In [0]:
percent_missing = market_dt.isnull().sum() * 100 / len(market_dt)
missing_value_df = pd.DataFrame({'column_name': market_dt.columns,
                                 'percent_missing': percent_missing})


In [0]:
missing_value_df

,column_name,percent_missing
time,time,0.000000
assetCode,assetCode,0.000000
assetName,assetName,0.000000
volume,volume,0.000000
close,close,0.000000
open,open,0.000000
returnsClosePrevRaw1,returnsClosePrevRaw1,0.000000
returnsOpenPrevRaw1,returnsOpenPrevRaw1,0.000000
returnsClosePrevMktres1,returnsClosePrevMktres1,0.090131
returnsOpenPrevMktres1,returnsOpenPrevMktres1,0.090461


In [0]:
market_dt.drop(['returnsClosePrevMktres10','returnsOpenPrevMktres10'], axis=1, inplace=True)
market_null_list = market_dt.columns[market_dt.isna().any()].tolist()
imp = Imputer(missing_values='NaN', strategy='median', axis=0)
imp = imp.fit(market_dt[market_null_list])
market_dt[market_null_list] = imp.transform(market_dt[market_null_list])




In [0]:
market_dt.shape

(2423150, 19)

In [0]:
news_dt = pd.read_csv('/content/drive/My Drive/ML/FinalProject/dataset/news_market_train_uni1_v1.csv')

In [0]:
def label_cat(df, col):
    return df[col].astype('category').cat.as_ordered()

def bin_encode(df, cols_to_bin):
    ce_bin = ce.BinaryEncoder(cols = cols_to_bin)
    return ce_bin.fit_transform(df)

# Find all features with NaN's
def get_null_features(df):
    return df.columns[df.isna().any()].tolist()

# Returns either month, day or year of date -> expedites feature engineering 
def get_date_feature(df, col, date_type):
    if (date_type == 'year'):
        return pd.to_datetime(df[col]).dt.year
    elif (date_type == 'month'):
        return pd.to_datetime(df[col]).dt.month
    elif (date_type == 'quarter'):
        return pd.to_datetime(df[col]).dt.quarter
    else:
        return pd.to_datetime(df[col]).dt.dayofweek

# Helper function to print accuracy
def print_accuracy(model, x, y, x_val, y_val):
    print("Training Accuracy Score: ", accuracy_score(model.predict(x), y))
    print("Validation Accuracy Score: ", accuracy_score(model.predict(x_val), y_val))
    
# Helper function to get feature importances
def get_feature_imp(model, x):
    return pd.DataFrame(model.feature_importances_,
                                   index = x.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

# Calculate the RSI
def RSI(series, period):
    delta = series.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
    d = d.drop(d.index[:(period-1)])
    rs = u.ewm(com=period-1, adjust=False).mean() / d.ewm(com=period-1, adjust=False).mean()
    return 100 - 100 / (1 + rs)

# 80%, 20%
def temploral_split(df, labels):
    X_train, X_val = np.split(df, [int(.8*len(df))])
    y_t, y_v = np.split(labels, [int(.8*len(labels))])
    
    y_train = y_t > 0    
    y_val = y_v > 0
    
    return [X_train, y_train, X_val, y_val]

# Join 2 dataframes
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

# Random search for temporal data
def random_search(model, param_grid, X_train, y_train, X_val, y_val):
    best_acc = 0
    for i in range(10):
        params = {k: np.random.choice(v) for k, v in param_grid.items()}
        params['n_jobs'] = -1
        new_model = model(**params)
        new_model.fit(X_train, y_train)
        acc = accuracy_score(new_model.predict(X_val), y_val)

        if acc > best_acc or best_acc == 0:
            best_acc = acc
            best_params = params
    print(f'Best Score: {best_acc}')
    print(f'Best Paramas: {best_params}')
    return best_params, best_acc


In [0]:
# Prep market data
# input: market dataframe
# output: preprocessed and feature engineered dataframe
def prep_market(df):
    
    
    #df['time'] = df.time.dt.strftime("%Y%m%d").astype(int)
    
    # Create year, month, day features
    df['month'] = get_date_feature(df, 'time', 'month')
    df['day'] = get_date_feature(df, 'time', 'day')
    df['quarter'] = get_date_feature(df, 'time', 'quarter')
    
    # Some feature engineering -> moving averages
    for n in [14, 30, 50, 200]:
        # Create the moving averages
        df['close_ma' + str(n)] = df.groupby('assetCode')['close'].transform(lambda x: x.rolling(window=n).mean())
    
    # Create RSI -> only 14 was useful during feature exploration
    #df['rsi14'] = RSI(df['close'], 14)
    
    # 10 day pct change in volume
    df['vol_pct_change'] = df.groupby('assetCode')['volume'].transform(lambda x: x.pct_change())
    df['vol_pct_10'] = df.groupby('assetCode')['vol_pct_change'].transform(lambda x : x.rolling(window=10).mean())
    
    # Log Returns
    df['log_ret'] = df.groupby('assetCode')['close'].transform(lambda x: np.log(x) - np.log(x.shift()))

    # Momentum
    df['mom1'] = df.groupby('assetCode')['close'].transform(lambda x: x.pct_change(periods=1))
    df['mom2'] = df.groupby('assetCode')['close'].transform(lambda x: x.pct_change(periods=2))
    df['mom3'] = df.groupby('assetCode')['close'].transform(lambda x: x.pct_change(periods=3))
    df['mom4'] = df.groupby('assetCode')['close'].transform(lambda x: x.pct_change(periods=4))
    df['mom5'] = df.groupby('assetCode')['close'].transform(lambda x: x.pct_change(periods=5))
    df['mom5'] = df.groupby('assetCode')['close'].transform(lambda x: x.pct_change(periods=7))
    df['mom10'] = df.groupby('assetCode')['close'].transform(lambda x: x.pct_change(periods=10))

    # Volatility
    df['volatility_50'] = df.groupby('assetCode')['log_ret'].transform(lambda x: x.rolling(window=50, min_periods=50, center=False).std())
    df['volatility_31'] = df.groupby('assetCode')['log_ret'].transform(lambda x: x.rolling(window=31, min_periods=31, center=False).std())
    df['volatility_15'] = df.groupby('assetCode')['log_ret'].transform(lambda x: x.rolling(window=15, min_periods=15, center=False).std())
    window_autocorr = 50

    #market_dt['autocorr_1'] = market_dt['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=1), raw=False)
    #market_dt['autocorr_2'] = market_dt['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=2), raw=False)
    #market_dt['autocorr_3'] = market_dt['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=3), raw=False)
    #market_dt['autocorr_4'] = market_dt['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=4), raw=False)
    #market_dt['autocorr_5'] = market_dt['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=5), raw=False)

    # drop 200 rows -> this is because of the moving average calculations
    df.dropna(inplace=True)
    _ = gc.collect()
    
    # These were identified in part 2 -> returnsOpenNextMktres10 is the dependant var
    train_cols = ['returnsClosePrevRaw10',
                  'returnsClosePrevMktres10',  
                  'close',
                  #'rsi14', 
                  'assetCode',
                  'month', 
                  'close_ma14',
                  'close_ma200',
                  'close_ma30',
                  'volume',
                  'close_ma50',
                  'returnsClosePrevMktres1', 
                  'returnsClosePrevRaw1',
                  'returnsOpenPrevRaw1',
                  'vol_pct_10',
                  'day',
                  'quarter',
                  'time',
                  'mom1',
                  'mom2',
                  'mom3',
                  'mom4',
                  'mom5',
                  'mom10',
                  'volatility_50',
                  'volatility_31',
                  'volatility_15'
                   ]
    
    # This is for the final trainin set contidion
    if 'returnsOpenNextMktres10' in df.columns:
        train_cols = train_cols + ['returnsOpenNextMktres10']
    
    df = df[train_cols]
    
    return df



##Process news

In [0]:
def preprocess_news(news_train):
    drop_list = [
        'audiences', 'subjects', 'assetName',
        'headline', 'firstCreated', 'sourceTimestamp',
    ]
    news_train.drop(drop_list, axis=1, inplace=True)
    
    # Factorize categorical columns
    for col in ['headlineTag', 'provider', 'sourceId']:
        news_train[col], uniques = pd.factorize(news_train[col])
        del uniques
    
    # Remove {} and '' from assetCodes column
    news_train['assetCodes'] = news_train['assetCodes'].apply(lambda x: x[1:-1].replace("'", ""))
    return news_train

news_dt = preprocess_news(news_dt)

#Unstack news

In [0]:
def unstack_asset_codes(news_train):
    codes = []
    indexes = []
    for i, values in news_train['assetCodes'].iteritems():
        explode = values.split(", ")
        codes.extend(explode)
        repeat_index = [int(i)]*len(explode)
        indexes.extend(repeat_index)
    index_df = pd.DataFrame({'news_index': indexes, 'assetCode': codes})
    del codes, indexes
    gc.collect()
    return index_df

index_df = unstack_asset_codes(news_dt)
index_df.head()

,news_index,assetCode
0,0,0857.HK
1,0,0857.F
2,0,0857.DE
3,0,PTR.N
4,1,STA.N


##merge news on index

In [0]:
def merge_news_on_index(news_train, index_df):
    news_train['news_index'] = news_train.index.copy()

    # Merge news on unstacked assets
    news_unstack = index_df.merge(news_train, how='left', on='news_index')
    news_unstack.drop(['news_index', 'assetCodes'], axis=1, inplace=True)
    return news_unstack

news_unstack = merge_news_on_index(news_dt, index_df)
del news_dt, index_df
gc.collect()
news_unstack.head(3)

,assetCode,time,sourceId,urgency,takeSequence,provider,bodySize,companyCount,headlineTag,marketCommentary,sentenceCount,wordCount,firstMentionSentence,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D
0,0857.HK,2007-01-01 04:29:32+00:00,0,3,1,0,1438,1,-1,False,11,275,6,0.235702,-1,0.500739,0.419327,0.079934,73,0,0,0,0,0,0,0,3,6,7
1,0857.F,2007-01-01 04:29:32+00:00,0,3,1,0,1438,1,-1,False,11,275,6,0.235702,-1,0.500739,0.419327,0.079934,73,0,0,0,0,0,0,0,3,6,7
2,0857.DE,2007-01-01 04:29:32+00:00,0,3,1,0,1438,1,-1,False,11,275,6,0.235702,-1,0.500739,0.419327,0.079934,73,0,0,0,0,0,0,0,3,6,7


#Group by date and asset

In [0]:
def group_news(news_frame):
    news_frame['date'] = pd.to_datetime(news_frame.time).dt.date  # Add date column
    
    aggregations = ['mean']
    gp = news_frame.groupby(['assetCode', 'date']).agg(aggregations)
    gp.columns = pd.Index(["{}_{}".format(e[0], e[1]) for e in gp.columns.tolist()])
    gp.reset_index(inplace=True)
    # Set datatype to float32
    float_cols = {c: 'float32' for c in gp.columns if c not in ['assetCode', 'date']}
    return gp.astype(float_cols)

news_agg = group_news(news_unstack)
del news_unstack; gc.collect()
news_agg.head(3)

,assetCode,date,sourceId_mean,urgency_mean,takeSequence_mean,provider_mean,bodySize_mean,companyCount_mean,headlineTag_mean,marketCommentary_mean,sentenceCount_mean,wordCount_mean,firstMentionSentence_mean,relevance_mean,sentimentClass_mean,sentimentNegative_mean,sentimentNeutral_mean,sentimentPositive_mean,sentimentWordCount_mean,noveltyCount12H_mean,noveltyCount24H_mean,noveltyCount3D_mean,noveltyCount5D_mean,noveltyCount7D_mean,volumeCounts12H_mean,volumeCounts24H_mean,volumeCounts3D_mean,volumeCounts5D_mean,volumeCounts7D_mean
0,0005.HK,2007-01-02,87.000000,3.0,1.2,0.6,1676.199951,3.200000,2.0,0.000000,10.600000,272.799988,2.8,0.644895,0.2,0.055284,0.642587,0.302128,122.800003,0.6,0.6,0.6,1.6,1.8,2.200000,2.200000,2.200000,5.600000,9.800000
1,0005.HK,2007-01-03,499.700012,3.0,1.2,0.6,3888.500000,3.300000,-0.6,0.600000,59.500000,736.299988,3.5,0.319020,0.6,0.178223,0.291784,0.529992,269.000000,0.4,0.4,0.4,0.4,0.6,5.300000,9.200000,10.900000,13.300000,16.400000
2,0005.HK,2007-01-04,1451.666626,3.0,1.0,0.0,2400.333252,1.666667,11.0,0.333333,36.666668,482.000000,3.0,0.512449,1.0,0.161406,0.351436,0.487159,216.000000,0.0,0.0,0.0,0.0,0.0,1.666667,8.666667,19.666666,19.666666,23.333334


In [0]:

market_dt = prep_market(market_dt)


In [0]:
market_dt['date'] = pd.to_datetime(market_dt.time).dt.date
df = market_dt.merge(news_agg, how='left', on=['assetCode', 'date'])
del market_dt, news_agg
gc.collect()
df.head(3)

,returnsClosePrevRaw10,returnsClosePrevMktres10,close,assetCode,month,close_ma14,close_ma200,close_ma30,volume,close_ma50,returnsClosePrevMktres1,returnsClosePrevRaw1,returnsOpenPrevRaw1,vol_pct_10,day,quarter,time,mom1,mom2,mom3,mom4,mom5,mom10,volatility_50,volatility_31,volatility_15,returnsOpenNextMktres10,date,sourceId_mean,urgency_mean,takeSequence_mean,provider_mean,bodySize_mean,companyCount_mean,headlineTag_mean,marketCommentary_mean,sentenceCount_mean,wordCount_mean,firstMentionSentence_mean,relevance_mean,sentimentClass_mean,sentimentNegative_mean,sentimentNeutral_mean,sentimentPositive_mean,sentimentWordCount_mean,noveltyCount12H_mean,noveltyCount24H_mean,noveltyCount3D_mean,noveltyCount5D_mean,noveltyCount7D_mean,volumeCounts12H_mean,volumeCounts24H_mean,volumeCounts3D_mean,volumeCounts5D_mean,volumeCounts7D_mean
0,-0.071913,-0.035249,34.20,A.N,11,35.282143,36.030950,36.513667,1838001.0,36.577000,0.014433,0.008552,0.009786,0.042331,2,4,2007-11-14 22:00:00+00:00,0.008552,0.020591,0.024259,-0.022298,-0.041748,-0.071913,0.013705,0.015841,0.017991,0.123976,2007-11-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.050117,0.089347,35.83,AAP.N,11,35.106429,37.500900,34.449333,2167800.0,34.145800,0.007036,0.003923,0.029547,0.286085,2,4,2007-11-14 22:00:00+00:00,0.003923,0.035849,0.019056,-0.006103,0.019346,0.050117,0.018969,0.019905,0.017337,0.011278,2007-11-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.125507,0.046104,166.11,AAPL.O,11,179.784729,123.557732,174.972660,51755984.0,162.809776,-0.008777,-0.022652,0.081753,0.092543,2,4,2007-11-14 22:00:00+00:00,-0.022652,0.080320,0.004475,-0.053322,-0.107799,-0.125507,0.029006,0.034568,0.042136,0.118857,2007-11-14,282625.9375,2.857143,1.214286,0.0,2690.142822,4.857143,7.357143,0.142857,19.357143,492.928558,6.642857,0.419135,-0.071429,0.327014,0.298634,0.374352,171.428574,0.785714,0.785714,0.785714,0.785714,0.785714,27.142857,50.42857,66.5,74.85714,97.35714
